## Initialize and Load Claims

In [5]:
from pathlib import Path

data_dir = Path('/mnt/data/factcheck/tweets/similarity_web_data')

In [4]:
import json

def load_dict(flnm):
    with open(flnm, 'rt') as f:
        data = json.load(f)
    data = {int(k): v for k, v in data.items()}
    return data

all_to_text = load_dict(data_dir / 'all_text.json')
claims = list(all_to_text.values())

print(len(claims))

203057


## Lemmatize Claims Using Morphodita API

In [ ]:
import requests
import json

def lemmatized(text):
    url = 'http://lindat.mff.cuni.cz/services/morphodita/api/analyze'
    params = {
        'data': text,
        'output': 'json',
        'convert_tagset': 'strip_lemma_id'
    }
    r = requests.post(url=url, params=params) 
    response = r.json()['result']
    return [word['analyses'][0]['lemma']
            for sentence in response for word in sentence]

lemmatized('Meziroční nárůst investic do dopravní infrastruktury o třetinu, akcelerace strategických úseků dálnic, bezpečnostní opatření na železnici, el. dálniční známky nebo jednotné jízdenky pro všechny žel. dopravce. Témata, která jsem dnes představil na Dopravní konferenci Pardubice.')

In [8]:
import time
import json

path = Path('/mnt/data/factcheck/tweets/clustering/lemmatized')

def convert(data):
    lemmas = []
    for i, c in enumerate(data):
        print(f'\r{i}', end='')
        lemmas.append(lemmatized(c))
        time.sleep(0.1)
    print()
    return lemmas
        
def flight(num, data):
    sub = data[num*10000: (num+1)*10000]
    res = convert(sub)
    with open(path / f'lemmatized{num:02d}.json', 'wt') as file:
        json.dump(res, file, ensure_ascii=False)

In [9]:
flight(0, claims)

9999


In [18]:
with open(path / 'lemmatized00.json', 'rt') as file:
    s = json.load(file)
print(len(s))

10000


In [ ]:
for i in range(1, 21):
    flight(i, claims)

## Join Fleets Together

In [1]:
import json
from pathlib import Path

path = Path('/mnt/data/factcheck/tweets/clustering/lemmatized')

all_data = []

for num in range(21):
    with open(path / f'lemmatized{num:02d}.json', 'rt') as file:
        loaded = json.load(file)
    all_data.extend(loaded)
    
print(len(all_data))

path = Path('/mnt/data/factcheck/tweets/clustering')
with open(path / 'lemmatized_all.json', 'wt') as file:
    json.dump(all_data, file, ensure_ascii=False)

203057
